In [132]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import requests

from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
from xgboost import XGBClassifier
from tqdm import tqdm
from time import sleep

from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, hamming_loss
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from tensorflow.keras import layers, Sequential, callbacks, optimizers, utils, models, applications
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# folder_path = '/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/'
# dataframes = []
# for file_name in os.listdir(folder_path):
#     if file_name.endswith('.csv'):
#         file_path = os.path.join(folder_path, file_name)
#         df = pd.read_csv(file_path)
#         dataframes.append(df)
# merged_df = pd.concat(dataframes, ignore_index=True)

In [3]:
merged_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/long_sypnosis.csv")

merged_df.shape

(4624, 6)

In [4]:
merged_df.head(5)

,Unnamed: 0,movie,imdb_id,genre,image_url,sypnosis
0,0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [5]:
merged_df = merged_df.drop(columns=['Unnamed: 0'])

In [6]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(4624, 5)

In [7]:
merged_df.head(5)

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."


In [8]:
merged_df["sypnosis"].notna().sum()

4624

In [9]:
merged_df

,movie,imdb_id,genre,image_url,sypnosis
0,Ramayana: The Legend of Prince Rama,tt0259534,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...
1,The Dark Knight,tt0468569,"['Action', ' Crime', ' Drama']",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...
2,The Lord of the Rings: The Return of the King,tt0167260,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...
3,Spider-Man: Across the Spider-Verse,tt9362722,"['Animation', ' Action', ' Adventure']",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,..."
4,The Message,tt0075143,"['Action', ' Adventure', ' Biography']",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive..."
...,...,...,...,...,...
4619,Bad Girls,tt0109198,"['Action', ' Adventure', ' Drama']",http://img.omdbapi.com/?i=tt0109198&h=600&apik...,Four prostitutes join together to travel the O...
4620,Renegade,tt0276830,"['Adventure', ' Fantasy', ' Western']",http://img.omdbapi.com/?i=tt0276830&h=600&apik...,"In 1870, after a brutal run-in with an outlaw ..."
4621,Wild Wild West,tt0120891,"['Action', ' Comedy', ' Sci-Fi']",http://img.omdbapi.com/?i=tt0120891&h=600&apik...,The two best special agents in the Wild West m...
4622,The Ridiculous 6,tt2479478,"['Action', ' Adventure', ' Comedy']",http://img.omdbapi.com/?i=tt2479478&h=600&apik...,An outlaw who was raised by Native Americans d...


In [10]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

In [11]:
def cleaning(sentence):
    if isinstance(sentence, str):
    # Basic cleaning
        sentence = sentence.strip() ## remove whitespaces
        sentence = sentence.lower() ## lowercase 
        sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

        # Advanced cleaning
        for punctuation in string.punctuation:
            sentence = sentence.replace(punctuation, '') ## remove punctuation

        tokenized_sentence = word_tokenize(sentence) ## tokenize 
        stop_words = set(stopwords.words('english')) ## define stopwords

        tokenized_sentence_cleaned = [ ## remove stopwords
            w for w in tokenized_sentence if not w in stop_words
        ]

        lemmatized = [
            WordNetLemmatizer().lemmatize(word, pos = "v") 
            for word in tokenized_sentence_cleaned
        ]

        cleaned_sentence = ' '.join(word for word in lemmatized)

        return cleaned_sentence

In [12]:
# merged_df["clean_plot"] = merged_df["plot"].apply(cleaning)

In [13]:
merged_df["clean_sypnosis"] = merged_df["sypnosis"].apply(cleaning)

In [14]:
merged_df["clean_title"] = merged_df["movie"].apply(cleaning)

In [15]:
merged_df["genre"] = (
    merged_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [16]:
merged_df.shape

(4624, 7)

In [17]:
merged_df = merged_df.dropna()

In [18]:
merged_df.shape

(4624, 7)

In [19]:
#check the genre we have
merged_df["genre"].explode().value_counts()

Drama        2824
Comedy       1495
Adventure    1045
Action        994
Crime         806
Romance       647
Biography     591
Thriller      544
Horror        511
Mystery       504
Animation     405
Fantasy       387
Sci-Fi        360
Family        312
History       298
Music         261
War           201
Sport         187
Musical       106
Western       104
Film-Noir      59
Name: genre, dtype: int64

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(merged_df["genre"])

# transform target variable
y = multilabel_binarizer.transform(merged_df['genre'])

In [21]:
genre_names = multilabel_binarizer.classes_

# Adding 
for i in range(len(genre_names)):
    merged_df[f"{genre_names[i]}"] = y[:,i]

merged_df.shape

(4624, 28)

In [22]:
merged_df.head(5)

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Ramayana: The Legend of Prince Rama,tt0259534,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt0259534&h=600&apik...,An anime adaptation of the Hindu epic the Rama...,anime adaptation hindu epic ramayana lord ram ...,ramayana legend prince rama,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,The Dark Knight,tt0468569,"[Action, Crime, Drama]",http://img.omdbapi.com/?i=tt0468569&h=600&apik...,When the menace known as the Joker wreaks havo...,menace know joker wreak havoc chaos people got...,dark knight,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Spider-Man: Across the Spider-Verse,tt9362722,"[Animation, Action, Adventure]",http://img.omdbapi.com/?i=tt9362722&h=600&apik...,"Miles Morales catapults across the Multiverse,...",miles morales catapult across multiverse encou...,spiderman across spiderverse,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,The Message,tt0075143,"[Action, Adventure, Biography]",http://img.omdbapi.com/?i=tt0075143&h=600&apik...,"The story of prophet ""Muhammad"" and the delive...",story prophet muhammad delivery message god allah,message,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# from imblearn.under_sampling import RandomUnderSampler
# from sklearn.datasets import make_classification
# from collections import Counter

# # Generate a sample imbalanced dataset
# X, y = make_classification(n_classes=2, class_sep=2, weights=[0.1, 0.9], n_informative=3, n_redundant=1, flip_y=0, n_features=20, n_clusters_per_class=1, n_samples=1000, random_state=10)

# # Count the class distribution before undersampling
# print("Class distribution before undersampling:", Counter(y))

# # Apply random undersampling
# rus = RandomUnderSampler(random_state=42)
# X_resampled, y_resampled = rus.fit_resample(X, y)

# # Count the class distribution after undersampling
# print("Class distribution after undersampling:", Counter(y_resampled))

In [24]:
# X = merged_df[["clean_sypnosis","clean_title"]]
# y = merged_df.drop(columns=["movie","imdb_id","genre","image_url","sypnosis","clean_sypnosis","clean_title"])

In [25]:
condition1 =  (merged_df['Animation'] == 1) | (merged_df['Music'] == 1)  | (merged_df['Biography'] == 1) | (merged_df['Sport'] == 1) |(merged_df['Musical'] == 1) |(merged_df['War'] == 1)|(merged_df['Western'] == 1)|(merged_df['History'] == 1)|(merged_df['Film-Noir'] == 1)|(merged_df['Family'] == 1)

rows_to_drop1 = merged_df[condition1].index
test_df = merged_df.drop(rows_to_drop1)

test_df["genre"].explode().value_counts()

Drama        1446
Comedy        744
Action        703
Crime         608
Adventure     493
Horror        490
Thriller      480
Mystery       475
Romance       418
Sci-Fi        344
Fantasy       284
Name: genre, dtype: int64

In [26]:
condition2 = (test_df['Comedy'] == 1)  & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)
rows_to_drop2 = test_df[condition2].index
test_df2 = test_df.drop(rows_to_drop2)

   
test_df2["genre"].explode().value_counts()

Drama        1267
Action        586
Crime         497
Horror        490
Thriller      480
Mystery       475
Romance       418
Comedy        383
Adventure     359
Sci-Fi        344
Fantasy       284
Name: genre, dtype: int64

In [27]:
condition3 = (test_df2['Action'] == 1)  & (test_df2['Crime'] == 1)  & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)
# & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Biography'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Animation'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)& (test_df['History'] != 1)& (test_df['Family'] != 1)& (test_df['Music'] != 1)
rows_to_drop3 = test_df2[condition3].index
test_df3= test_df2.drop(rows_to_drop3)

   
test_df3["genre"].explode().value_counts()

/tmp/ipykernel_639/3571784835.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rows_to_drop3 = test_df2[condition3].index


Drama        1174
Horror        490
Action        485
Thriller      480
Mystery       475
Romance       418
Crime         396
Comedy        383
Adventure     351
Sci-Fi        344
Fantasy       284
Name: genre, dtype: int64

In [28]:
test_df3.columns

Index(['movie', 'imdb_id', 'genre', 'image_url', 'sypnosis', 'clean_sypnosis',
       'clean_title', 'Action', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [29]:
no_drama_df = test_df3.drop(columns=["Drama", "Biography", "Family","Film-Noir","History","Music","Musical","Sport", "War", "Western", "Animation"])

In [30]:
no_drama_df.columns

Index(['movie', 'imdb_id', 'genre', 'image_url', 'sypnosis', 'clean_sypnosis',
       'clean_title', 'Action', 'Adventure', 'Comedy', 'Crime', 'Fantasy',
       'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller'],
      dtype='object')

In [31]:
condition_null = (no_drama_df['Action'] == 1) | (no_drama_df['Adventure'] == 1) | (no_drama_df['Comedy'] == 1)| (no_drama_df['Crime'] == 1)| (no_drama_df['Fantasy'] == 1)| (no_drama_df['Horror'] == 1)| (no_drama_df['Mystery'] == 1)| (no_drama_df['Romance'] == 1)| (no_drama_df['Sci-Fi'] == 1)| (no_drama_df['Thriller'] == 1)
no_drama_df1 =no_drama_df.loc[no_drama_df[condition_null].index]

In [32]:
no_drama_df1

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
2,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,0,0,0,0,0,0,0
5,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,A thief who steals corporate secrets through t...,thief steal corporate secrets use dreamsharing...,inception,1,1,0,0,0,0,0,0,1,0
6,The Lord of the Rings: The Fellowship of the Ring,tt0120737,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0120737&h=600&apik...,A meek Hobbit from the Shire and eight compani...,meek hobbit shire eight companion set journey ...,lord ring fellowship ring,1,1,0,0,0,0,0,0,0,0
7,The Lord of the Rings: The Two Towers,tt0167261,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167261&h=600&apik...,While Frodo and Sam edge closer to Mordor with...,frodo sam edge closer mordor help shifty gollu...,lord ring two tower,1,1,0,0,0,0,0,0,0,0
8,Kill Bill: The Whole Bloody Affair,tt6019206,"[Action, Crime, Thriller]",http://img.omdbapi.com/?i=tt6019206&h=600&apik...,The Bride must kill her ex-boss and lover Bill...,bride must kill exboss lover bill betray wed r...,kill bill whole bloody affair,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,Chaos Walking,tt2076822,"[Action, Adventure, Fantasy]",http://img.omdbapi.com/?i=tt2076822&h=600&apik...,Two unlikely companions embark on a perilous a...,two unlikely companion embark perilous adventu...,chaos walk,1,1,0,0,1,0,0,0,0,0
4614,The Dark Tower,tt1648190,"[Action, Adventure, Fantasy]",http://img.omdbapi.com/?i=tt1648190&h=600&apik...,A boy haunted by visions of a dark tower from ...,boy haunt visions dark tower parallel reality ...,dark tower,1,1,0,0,1,0,0,0,0,0
4619,Bad Girls,tt0109198,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0109198&h=600&apik...,Four prostitutes join together to travel the O...,four prostitute join together travel old west ...,bad girls,1,1,0,0,0,0,0,0,0,0
4621,Wild Wild West,tt0120891,"[Action, Comedy, Sci-Fi]",http://img.omdbapi.com/?i=tt0120891&h=600&apik...,The two best special agents in the Wild West m...,two best special agents wild west must save pr...,wild wild west,1,0,1,0,0,0,0,0,1,0


In [33]:
no_drama_df1.reset_index(inplace=True)

In [34]:
no_drama_df1 = no_drama_df1.drop(columns=["index"])

In [35]:
no_drama_df1.head(3)

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,0,0,0,0,0,0,0
1,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,A thief who steals corporate secrets through t...,thief steal corporate secrets use dreamsharing...,inception,1,1,0,0,0,0,0,0,1,0
2,The Lord of the Rings: The Fellowship of the Ring,tt0120737,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0120737&h=600&apik...,A meek Hobbit from the Shire and eight compani...,meek hobbit shire eight companion set journey ...,lord ring fellowship ring,1,1,0,0,0,0,0,0,0,0


In [36]:
no_drama_df1.to_csv("filtered.csv")

In [37]:
no_drama_df1["genre"].explode().value_counts()

Drama        1112
Horror        490
Action        485
Thriller      480
Mystery       475
Romance       418
Crime         396
Comedy        383
Adventure     351
Sci-Fi        344
Fantasy       284
Name: genre, dtype: int64

In [38]:
# condition = (merged_df['Drama'] == 1) & (merged_df['Comedy'] == 1)
# rows_to_drop = merged_df[condition].head(500).index
# balancing_df = merged_df.drop(rows_to_drop)
# balancing_df.shape
# balancing_df["genre"].explode().value_counts()

In [39]:
# condition1 = (merged_df['Drama'] == 1)&(merged_df['Romance'] != 1)& (merged_df['Sci-Fi'] != 1)& (merged_df['Fantasy'] != 1)
# # & (merged_df['Crime'] != 1)& (merged_df['Romance'] != 1)& (merged_df['Thriller'] != 1)& (merged_df['Biography'] != 1)& (merged_df['Horror'] != 1)& (merged_df['Mystery'] != 1)& (merged_df['Animation'] != 1)& (merged_df['Sci-Fi'] != 1)& (merged_df['Fantasy'] != 1)& (merged_df['History'] != 1)& (merged_df['Family'] != 1)& (merged_df['Music'] != 1)
# # & (merged_df['War'] == 1)& (merged_df['Sport'] == 1)& (merged_df['Western'] == 1)& (merged_df['Musical'] == 1)& (merged_df['Film-Noir'] == 1)
# # & (merged_df['Adventure'] == 1)& (merged_df['Action'] != 1)
# # & (merged_df['Comedy'] != 1)

# rows_to_drop1 = merged_df[condition1].index
# # .head(1000).index
# test_df = merged_df.drop(rows_to_drop1)


   
# test_df["genre"].explode().value_counts()

In [40]:
# condition2 = (test_df['Comedy'] == 1)  & (test_df['Adventure'] == 1)
# # & (test_df['Romance'] != 1)& (test_df['Thriller'] != 1)& (test_df['Biography'] != 1)& (test_df['Horror'] != 1)& (test_df['Mystery'] != 1)& (test_df['Animation'] != 1)& (test_df['Sci-Fi'] != 1)& (test_df['Fantasy'] != 1)& (test_df['History'] != 1)& (test_df['Family'] != 1)& (test_df['Music'] != 1)
# rows_to_drop2 = test_df[condition2].index
# test_df2 = test_df.drop(rows_to_drop2)

   
# test_df2["genre"].explode().value_counts()

In [41]:
# condition3 = (test_df['Music'] == 1) | (test_df['Biography'] == 1) | (test_df['Sport'] == 1) |(test_df['Musical'] == 1) |(test_df['War'] == 1)|(test_df['Western'] == 1)|(test_df['History'] == 1)|(test_df['Film-Noir'] == 1)|(test_df['Family'] == 1)

# rows_to_drop3 = test_df[condition3].index
# test_df3 = test_df.drop(rows_to_drop3)

   
# test_df3["genre"].explode().value_counts()

In [42]:
# condition2 = (balancing_df['Drama'] == 1)
# rows_to_drop = balancing_df[condition2].head(500).index
# balancing_df2 = balancing_df.drop(rows_to_drop)
# balancing_df2["genre"].explode().value_counts()

In [43]:
no_drama_df1.head(5)

,movie,imdb_id,genre,image_url,sypnosis,clean_sypnosis,clean_title,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,The Lord of the Rings: The Return of the King,tt0167260,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167260&h=600&apik...,Gandalf and Aragorn lead the World of Men agai...,gandalf aragorn lead world men saurons army dr...,lord ring return king,1,1,0,0,0,0,0,0,0,0
1,Inception,tt1375666,"[Action, Adventure, Sci-Fi]",http://img.omdbapi.com/?i=tt1375666&h=600&apik...,A thief who steals corporate secrets through t...,thief steal corporate secrets use dreamsharing...,inception,1,1,0,0,0,0,0,0,1,0
2,The Lord of the Rings: The Fellowship of the Ring,tt0120737,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0120737&h=600&apik...,A meek Hobbit from the Shire and eight compani...,meek hobbit shire eight companion set journey ...,lord ring fellowship ring,1,1,0,0,0,0,0,0,0,0
3,The Lord of the Rings: The Two Towers,tt0167261,"[Action, Adventure, Drama]",http://img.omdbapi.com/?i=tt0167261&h=600&apik...,While Frodo and Sam edge closer to Mordor with...,frodo sam edge closer mordor help shifty gollu...,lord ring two tower,1,1,0,0,0,0,0,0,0,0
4,Kill Bill: The Whole Bloody Affair,tt6019206,"[Action, Crime, Thriller]",http://img.omdbapi.com/?i=tt6019206&h=600&apik...,The Bride must kill her ex-boss and lover Bill...,bride must kill exboss lover bill betray wed r...,kill bill whole bloody affair,1,0,0,1,0,0,0,0,0,1


## Split Data and Vectorize

In [44]:
X = no_drama_df1[["movie","clean_sypnosis"]]
y = no_drama_df1.drop(columns=["movie","imdb_id","genre","image_url","sypnosis","clean_sypnosis","clean_title"])

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [46]:
X_train.shape

(1347, 2)

In [47]:
X_train.head(5)

,movie,clean_sypnosis
176,From Dusk Till Dawn,two criminals hostages unknowingly seek tempor...
1127,Night of the Living Dead,unburied dead return life seek human victims s...
724,Department Q: The Absent One,murder young twin initially implicate group up...
1300,Scouts Guide to the Zombie Apocalypse,three scout eve last campout discover true mea...
1321,The Blob,alien lifeform consume everything path grow gr...


In [48]:
X_test.head(5)

,movie,clean_sypnosis
677,Death on the Nile,hercule poirot enjoy luxurious cruise nile new...
240,Constantine,supernatural exorcist demonologist john consta...
1627,The Edukators,three activists cobble together kidnap plot en...
1674,A Better Life,gardener east la struggle keep son away gang i...
598,"The Cook, the Thief, His Wife & Her Lover",le hollandais gourmet restaurant every night f...


In [49]:
y_train.shape 

(1347, 10)

In [50]:
y_train.head(5)

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
176,1,0,0,1,0,1,0,0,0,0
1127,0,0,0,0,0,1,0,0,0,0
724,0,0,0,1,0,0,1,0,0,0
1300,1,0,1,0,0,1,0,0,0,0
1321,0,0,0,0,0,1,0,0,1,0


In [51]:
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=0.15, max_features=10000)
# tf_idf_title_vectorizer = TfidfVectorizer(min_df=10, max_features=10000)
# tf_idf_vectorizer = TfidfVectorizer()

# Training it on the texts
X_train_vec = pd.DataFrame(tf_idf_vectorizer.fit_transform(X_train["clean_sypnosis"]).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
# X_train_title_vec = pd.DataFrame(tf_idf_title_vectorizer.fit_transform(X_train["clean_title"]).toarray(),
#                  columns = tf_idf_title_vectorizer.get_feature_names_out())
X_test_vec =  pd.DataFrame(tf_idf_vectorizer.transform(X_test["clean_sypnosis"]).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
# X_test_title_vec =  pd.DataFrame(tf_idf_title_vectorizer.transform(X_test["clean_title"]).toarray(),
#                  columns = tf_idf_title_vectorizer.get_feature_names_out())


In [52]:
# X_test_title_vec

In [53]:
# X_train_vec = pd.concat([X_train_plot_vec,X_train_title_vec], axis=1)

In [54]:
# X_train_vec

In [55]:
# X_train_vec.shape

In [56]:
# X_test_vec = pd.concat([X_test_plot_vec,X_test_title_vec], axis=1)

In [57]:
X_train_vec.head(5)

,abandon,able,accept,accident,across,act,action,actually,admit,age,...,work,world,would,wound,write,wrong,year,years,yet,young
0,0.000000,0.024316,0.0,0.033879,0.026098,0.027413,0.000000,0.029632,0.029355,0.0,...,0.000000,0.022613,0.000000,0.031354,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.064408,0.000000,0.0,0.000000,0.000000,0.000000,0.132134,0.000000,0.000000,0.0,...,0.036405,0.000000,0.043338,0.129577,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.083874
3,0.000000,0.000000,0.0,0.000000,0.021260,0.000000,0.000000,0.000000,0.047827,0.0,...,0.043055,0.000000,0.017085,0.000000,0.023945,0.051083,0.0,0.033317,0.000000,0.000000
4,0.000000,0.103117,0.0,0.000000,0.073785,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.044453,0.000000


In [58]:
# X_test_vec.shape

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score

## OneVsRest logistic regression

### Simple OneVsRest logistic regression

In [60]:
log = LogisticRegression()
model_test_1 = OneVsRestClassifier(log)

model_test_1.fit(X_train_vec, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [61]:
y_pred = model_test_1.predict(X_test_vec)

In [62]:
accuracy_score(y_test, y_pred)

0.11418685121107267

In [63]:
f1_score(y_test, y_pred, average="micro")

0.4561003420752565

In [64]:
results = pd.DataFrame(y_pred, columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0


In [65]:
# multilabel_binarizer.inverse_transform(y_pred)[1]

In [66]:
y_test[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
677,0,0,0,1,0,0,1,0,0,0
240,1,0,0,0,1,1,0,0,0,0
1627,0,0,0,0,0,0,0,1,0,0
1674,0,0,0,0,0,0,0,1,0,0
598,0,0,0,1,0,0,0,0,0,0


#### Grouping the topic

In [67]:
# from sklearn.decomposition import LatentDirichletAllocation

# # Instantiate the LDA 
# n_components = 7
# lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 500)

# # Fit the LDA on the vectorized documents
# lda_model.fit(X_train_vec)

In [68]:
# X_train_topic_mixture = lda_model.transform(X_train_vec)
# X_train_topic_df = pd.DataFrame(X_train_topic_mixture)
# X_train_topic_df

In [69]:
# X_test_topic_mixture = lda_model.transform(X_test_vec)
# X_test_topic_df = pd.DataFrame(X_test_topic_mixture)
# X_test_topic_df

### Training OneVsRest using topic grouping as input

In [70]:
# model_test_2 = OneVsRestClassifier(log)
# model_test_2.fit(X_train_topic_df, y_train)

In [71]:
# y_pred_2 = model_test_2.predict(X_test_topic_df)
# accuracy_score(y_test, y_pred_2)

In [72]:
# results = pd.DataFrame(y_pred_2, columns=["Action","Adventure","Animation","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
# results[:5]

In [73]:
# y_test[:5]

### Using both topic grouping as input + OneVsRest 

In [74]:
# model_test_3 = OneVsRestClassifier(log)
# model_test_3.fit(X_train_topic_df, y_train)
# model_test_3.fit(X_train_vec, y_train)

In [75]:
# y_pred_3 = model_test_2.predict(X_test_topic_df)
# accuracy_score(y_test, y_pred_3)

## SGD

In [76]:
sgd = SGDClassifier(random_state = 42)
sgd_model = OneVsRestClassifier(sgd)
sgd_model.fit(X_train_vec,y_train)

OneVsRestClassifier(estimator=SGDClassifier(random_state=42))

In [77]:
y_pred_sgd1 = sgd_model.predict(X_test_vec)

In [78]:
# multilabel_binarizer.inverse_transform(y_pred_sgd1)[0:10]

In [79]:
accuracy_score(y_test, y_pred_sgd1)

0.1453287197231834

In [80]:
f1_score(y_test, y_pred_sgd1, average="micro")

0.5590321234876929

In [81]:
results = pd.DataFrame(y_pred_sgd1, columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[0:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,0,0,1,0,1,0,0,0,0
1,0,0,0,0,1,1,1,0,0,0
2,0,0,0,1,0,0,1,0,0,0
3,0,0,1,1,0,0,1,0,0,0
4,0,0,0,1,0,0,1,0,0,0


In [82]:
y_test[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
677,0,0,0,1,0,0,1,0,0,0
240,1,0,0,0,1,1,0,0,0,0
1627,0,0,0,0,0,0,0,1,0,0
1674,0,0,0,0,0,0,0,1,0,0
598,0,0,0,1,0,0,0,0,0,0


In [153]:
estimator = OneVsRestClassifier(SGDClassifier())
params = {
    "estimator__penalty": ["l1","l2"],
    "estimator__alpha": [0.000009, 0.00001,0.00002,0.00003, 0.00004, 0.00005, 0.0001],
}

grid_search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=params,
    scoring="f1_micro",  # Remove the list here
    n_iter=10,  # Number of parameter settings that are sampled
    random_state=42,  # For reproducibility
    n_jobs=-1  # Use all available CPU cores
)

grid_search.fit(X_train_vec, y_train)

RandomizedSearchCV(estimator=OneVsRestClassifier(estimator=SGDClassifier()),
                   n_jobs=-1,
                   param_distributions={'estimator__alpha': [9e-06, 1e-05,
                                                             2e-05, 3e-05,
                                                             4e-05, 5e-05,
                                                             0.0001],
                                        'estimator__penalty': ['l1', 'l2']},
                   random_state=42, scoring='f1_micro')

In [154]:
grid_search.best_params_

{'estimator__penalty': 'l1', 'estimator__alpha': 2e-05}

In [155]:
grid_search.best_score_

0.5460620160408131

In [156]:
best_SGD_model = OneVsRestClassifier(SGDClassifier(penalty="l1", alpha=0.00005))
best_SGD_model.fit(X_train_vec,y_train)
predictions_SGD = best_SGD_model.predict(X_test_vec)
print("Accuracy = ",accuracy_score(y_test,predictions_SGD))
print("F1 = ",f1_score(y_test,predictions_SGD, average="micro"))
print("Precision = ",precision_score(y_test,predictions_SGD,average="micro" ))
print("Hamming loss = ",hamming_loss(y_test,predictions_SGD))
print("\n")

Accuracy =  0.09688581314878893
F1 =  0.5386454183266932
Precision =  0.5360824742268041
Hamming loss =  0.20034602076124566




In [84]:
# multilabel_binarizer.inverse_transform(y_pred_sgd1)[2]

In [90]:
# test_df3[test_df3.clean_sypnosis == X_test.clean_sypnosis.iloc[2]]

## Random Forest Classifier

In [91]:
# rfc = RandomForestClassifier(random_state = 42)
# rfc_model = OneVsRestClassifier(rfc)

# rfc_model.fit(X_train_vec,y_train)

In [92]:
# y_pred_rfc1 = rfc_model.predict(X_test_vec)

In [93]:
# results = pd.DataFrame(y_pred_rfc1, columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
# results[4:5]

In [94]:
# no_drama_df[no_drama_df.clean_sypnosis == X_test.clean_sypnosis.iloc[4]]

In [95]:
# accuracy_score(y_test, y_pred_rfc1)

In [96]:
# f1_score(y_test, y_pred_rfc1, average="micro")

## Poly SVC

In [97]:
# X_train1, X_val1, y_train1, y_val1 = train_test_split(merged_df['clean_sypnosis'], y, test_size = 0.3, random_state = 0)
# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
# X_val_tfidf1 = tfidf_vectorizer.transform(X_val1)
# sgd = SGDClassifier()
# lr = LogisticRegression(C=1.0)
# svc = SVC(kernel='poly')
# nb = MultinomialNB()
# # One vs Restclassifier
# model = OneVsRestClassifier(svc)
# model.fit(X_train_tfidf1, y_train1)
# y_pred1 = model.predict(X_val_tfidf1)
# print(f'Accuracy score for SVC poly model is {accuracy_score(y_val1, y_pred1)}')
# print(f'F1 score for SVC poly model is {f1_score(y_val1, y_pred1, average="micro")}')

In [98]:
# sgd = SGDClassifier()
# lr = LogisticRegression(C=1.0)
# svc = SVC(kernel='poly')
# nb = MultinomialNB()
# # One vs Restclassifier
# model = OneVsRestClassifier(svc)
# model.fit(X_train_vec, y_train)
# y_pred_sgd2 = model.predict(X_test_vec)
# print(f'Accuracy score for SVC poly model is {accuracy_score(y_test, y_pred_sgd2)}')
# print(f'F1 score for SVC poly model is {f1_score(y_test, y_pred_sgd2, average="micro")}')

In [99]:
# X_train1, X_val1, y_train1, y_val1 = train_test_split(merged_df['clean_sypnosis'], y, test_size = 0.3, random_state = 0)
# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
# X_val_tfidf1 = tfidf_vectorizer.transform(X_val1)
# sgd = SGDClassifier()
# lr = LogisticRegression(C=1.0)
# svc = SVC(kernel='poly')
# nb = MultinomialNB()
# # One vs Restclassifier
# model = OneVsRestClassifier(svc)
# model.fit(X_train_tfidf1, y_train1)
# y_pred1 = model.predict(X_val_tfidf1)
# print(f'Accuracy score for SVC poly model is {accuracy_score(y_val1, y_pred1)}')
# print(f'F1 score for SVC poly model is {f1_score(y_val1, y_pred1)}')

## Label Powerset

In [100]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())
# train
classifier.fit(X_train_vec, y_train)
# predict
predictions_lp = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_lp))
print("F1 = ",f1_score(y_test,predictions_lp, average="micro"))
print("\n")

Accuracy =  0.23875432525951557
F1 =  0.5515527950310559




/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [101]:
predictions_lp.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [102]:
results = pd.DataFrame(predictions_lp.toarray(), columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0,0,0,1,0,0,1,0,0,0
1,1,1,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,0
3,0,0,1,0,0,0,0,1,0,0
4,0,0,0,1,0,0,1,0,0,0


In [103]:
y_test[:5]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
677,0,0,0,1,0,0,1,0,0,0
240,1,0,0,0,1,1,0,0,0,0
1627,0,0,0,0,0,0,0,1,0,0
1674,0,0,0,0,0,0,0,1,0,0
598,0,0,0,1,0,0,0,0,0,0


In [104]:
# no_drama_df[no_drama_df.clean_sypnosis == X_test.clean_sypnosis.iloc[4]]

In [105]:
# multilabel_binarizer.inverse_transform(predictions_lp)[0:10]

In [106]:
# condition = test_df3['clean_sypnosis'].isin(X_test['clean_sypnosis'][:10])
# filtered_df = test_df3[condition]
# filtered_df["genre"]

In [157]:
estimator = LabelPowerset(LogisticRegression())
params = {
    "classifier__penalty": ["l1","l2"],
    "classifier__C": range(5,9),
    "classifier__solver" : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}

grid_search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=params,
    scoring="f1_micro",  # Remove the list here
    n_iter=10,  # Number of parameter settings that are sampled
    random_state=42,  # For reproducibility
    n_jobs=-1  # Use all available CPU cores
)

grid_search.fit(X_train_vec, y_train)

/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted with

RandomizedSearchCV(estimator=LabelPowerset(classifier=LogisticRegression(),
                                           require_dense=[True, True]),
                   n_jobs=-1,
                   param_distributions={'classifier__C': range(5, 9),
                                        'classifier__penalty': ['l1', 'l2'],
                                        'classifier__solver': ['lbfgs',
                                                               'liblinear',
                                                               'newton-cg',
                                                               'newton-cholesky',
                                                               'sag', 'saga']},
                   random_state=42, scoring='f1_micro')

In [158]:
grid_search.best_params_

{'classifier__solver': 'liblinear',
 'classifier__penalty': 'l2',
 'classifier__C': 8}

In [159]:
grid_search.best_score_

0.5559607310110588

In [162]:
best_pl_model = LabelPowerset(LogisticRegression(penalty="l2", C=8, solver="liblinear"))
best_pl_model.fit(X_train_vec,y_train)
predictions_pl = best_pl_model.predict(X_test_vec)
print("Accuracy = ",accuracy_score(y_test,predictions_pl))
print("F1 = ",f1_score(y_test,predictions_pl, average="micro"))
print("Precision = ",precision_score(y_test,predictions_pl,average="micro" ))
print("Hamming loss = ",hamming_loss(y_test,predictions_pl))
print("\n")

Accuracy =  0.2491349480968858
F1 =  0.586622351474865
Precision =  0.6096718480138169
Hamming loss =  0.17214532871972318




/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


## Classifier Chains

In [107]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train_vec, y_train)
# predict
y_pred_cc = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ",accuracy_score(y_test,y_pred_cc))
print("\n")

Accuracy =  0.19896193771626297




In [108]:
y_pred_cc.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [109]:
results = pd.DataFrame(y_pred_cc.toarray(), columns=["Action","Adventure","Comedy","Crime","Fantasy","Horror","Mystery","Romance","Sci-Fi","Thriller"])
results[:3]

,Action,Adventure,Comedy,Crime,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [110]:
# multilabel_binarizer.inverse_transform(predictions)[0:3]

In [111]:
condition = no_drama_df['clean_sypnosis'].isin(X_test['clean_sypnosis'][:3])
filtered_df = no_drama_df[condition]
filtered_df["genre"]

477     [Action, Fantasy, Horror]
2232      [Crime, Drama, Mystery]
3986             [Drama, Romance]
Name: genre, dtype: object

In [163]:
estimator = ClassifierChain(LogisticRegression())
params = {
    "classifier__penalty": ["l1","l2"],
    "classifier__C": range(5,9),
    "classifier__solver" : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
}

grid_search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=params,
    scoring="f1_micro",  # Remove the list here
    n_iter=10,  # Number of parameter settings that are sampled
    random_state=42,  # For reproducibility
    n_jobs=-1  # Use all available CPU cores
)

grid_search.fit(X_train_vec, y_train)

/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/skmultilearn/problem_transform/cc.py", line 154, in fit
    self.classifiers_[label] = self.classifier.fit(self._ensure_input_format(
  File "/home/ying

RandomizedSearchCV(estimator=ClassifierChain(classifier=LogisticRegression(),
                                             require_dense=[True, True]),
                   n_jobs=-1,
                   param_distributions={'classifier__C': range(5, 9),
                                        'classifier__penalty': ['l1', 'l2'],
                                        'classifier__solver': ['lbfgs',
                                                               'liblinear',
                                                               'newton-cg',
                                                               'newton-cholesky',
                                                               'sag', 'saga']},
                   random_state=42, scoring='f1_micro')

In [164]:
grid_search.best_params_

{'classifier__solver': 'liblinear',
 'classifier__penalty': 'l2',
 'classifier__C': 8}

In [166]:
best_cc_model = ClassifierChain(LogisticRegression(penalty="l2", C=8, solver="liblinear"))
best_cc_model.fit(X_train_vec,y_train)
predictions_cc = best_cc_model.predict(X_test_vec)
print("Accuracy = ",accuracy_score(y_test,predictions_cc))
print("F1 = ",f1_score(y_test,predictions_cc, average="micro"))
print("Precision = ",precision_score(y_test,predictions_cc,average="micro" ))
print("Hamming loss = ",hamming_loss(y_test,predictions_cc))
print("\n")

Accuracy =  0.22491349480968859
F1 =  0.5817104149026249
Precision =  0.6172506738544474
Hamming loss =  0.17093425605536333




## Grid Search